In [3]:
import pandas as pd
import numpy as np
import janitor as jntr

characteristic_data = pd.read_csv("hego_data/hego_17_18_salary_by_characteristic.csv", skiprows = 13).clean_names()
institution_data = pd.read_csv("hego_data/hego_17_18_salary_by_institution.csv", skiprows = 13).clean_names()
subject_data = pd.read_csv("hego_data/hego_17_18_salary_by_subject.csv", skiprows = 13).clean_names()
uni_codes = pd.read_excel("hego_data/institution-and-campus-codes-2018-entry.xls").clean_names()